In [1]:
%pip install pandas

  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached numpy-2.2.5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached numpy-2.2.5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.1 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]2m3/4 [pandas]
Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import csv

ART_STYLES = [
    "Baroque", "Early_Renaissance", "High_Renaissance",
    "Mannerism_Late_Renaissance", "Northern_Renaissance", "Rococo"
]

def clean(text):
    return ' '.join(word.capitalize() for word in text.split('-'))

def extract_metadata(filepath):
    filename = os.path.basename(filepath)
    dirname = os.path.basename(os.path.dirname(filepath))
    name = os.path.splitext(filename)[0]  # Supprime .jpg etc.

    # Supprime les suffixes -1, (1), etc.
    name = name.replace('(', '-').replace(')', '')
    while name[-1].isdigit() and name[-2] in ('-', '_'):
        name = name.rsplit('-', 1)[0] if '-' in name else name.rsplit('_', 1)[0]

    # Séparer artiste et titre
    if '_' not in name:
        return None
    artist_part, title_part = name.split('_', 1)

    # Séparer les morceaux du titre
    parts = title_part.split('-')

    # Vérifie si le dernier bloc est une année
    year = ''
    if parts and parts[-1].isdigit() and len(parts[-1]) == 4:
        year = parts.pop()

    title_clean = clean('-'.join(parts))
    artist_clean = clean(artist_part)

    return {
        'style': dirname,
        'filename': filename,
        'artist': artist_clean,
        'title': title_clean,
        'year': year
    }

def scan_images(root_dir, output_csv):
    entries = []
    for style in ART_STYLES:
        style_path = os.path.join(root_dir, style)
        if not os.path.isdir(style_path):
            continue
        for fname in os.listdir(style_path):
            if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
                full_path = os.path.join(style_path, fname)
                meta = extract_metadata(full_path)
                if meta:
                    entries.append(meta)

    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['style', 'filename', 'artist', 'title', 'year']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')
        writer.writeheader()
        writer.writerows(entries)

    print(f"✅ CSV généré avec {len(entries)} entrées : {output_csv}")


In [6]:
dir_images = os.path.join(os.getcwd(), "images")
scan_images(dir_images, 'data.csv')

✅ CSV généré avec 120 entrées : data.csv


In [2]:
import pandas as pd

# Chargement du CSV généré précédemment
df = pd.read_csv('data.csv', delimiter=';')

# Génération des noms chiffrés
df['encrypted_name'] = ['image{:05d}.jpg'.format(i + 1) for i in range(len(df))]

# Affichage des 5 premiers pour vérification
df.head()

# Sauvegarde dans un nouveau CSV
df.to_csv('data_encrypted.csv', sep=';', index=False)

print(f"✅ Fichier sauvegardé avec {len(df)} images chiffrées dans 'data_encrypted.csv'")


✅ Fichier sauvegardé avec 120 images chiffrées dans 'data_encrypted.csv'


In [3]:
import os

dir_images = os.path.join(os.getcwd(), "images")

for idx, row in df.iterrows():
    old_path = os.path.join(dir_images, row['style'], row['filename'])
    new_path = os.path.join(dir_images, row['style'], row['encrypted_name'])
    if os.path.exists(old_path):
        os.rename(old_path, new_path)
    else:
        print(f"⚠️ Fichier introuvable : {old_path}")

print("✅ Renommage terminé.")

✅ Renommage terminé.


In [4]:
import pandas as pd

def js_escape(s):
    if pd.isna(s):
        return ""
    # Si s est un float entier (ex: 1628.0), le convertir en int puis en str
    if isinstance(s, float) and s.is_integer():
        s = int(s)
    return str(s).replace('"', '\\"')

df = pd.read_csv('data_encrypted.csv', delimiter=';')

images_js_entries = []

for _, row in df.iterrows():
    img_path = f"{row['style']}/{row['encrypted_name']}"
    entry = (
        '{'
        f'src:"images/{js_escape(img_path)}", '
        f'style:"{js_escape(row["style"])}", '
        f'artist:"{js_escape(row["artist"])}", '
        f'title:"{js_escape(row["title"])}", '
        f'year:"{js_escape(row["year"])}"'
        '}'
    )
    images_js_entries.append(entry)

js_code = "const images = [\n" + ",\n".join(images_js_entries) + "\n];"

with open("images_data.js", "w", encoding="utf-8") as f:
    f.write(js_code)

print("✅ Fichier 'images_data.js' généré.")


✅ Fichier 'images_data.js' généré.


In [4]:
import os
import random
import shutil

ART_STYLES = [
    "Baroque", "Early_Renaissance", "High_Renaissance",
    "Mannerism_Late_Renaissance", "Northern_Renaissance", "Rococo"
]

source_dir = '/home/phusson/Bureau/images_wikiart/'
dest_dir = f'{os.getcwd()}/images'

os.makedirs(dest_dir, exist_ok=True)

for style in ART_STYLES:
    source_dir_style = os.path.join(source_dir, style)
    dest_dir_style = os.path.join(dest_dir, style)
    os.makedirs(dest_dir_style, exist_ok=True)
    # Liste tous les fichiers dans source_dir
    files = [f for f in os.listdir(source_dir_style) if os.path.isfile(os.path.join(source_dir_style, f))]

    # Choisit 20 fichiers au hasard (ou moins si moins de 20)
    sample_files = random.sample(files, min(20, len(files)))

    for f in sample_files:
        shutil.copy2(os.path.join(source_dir_style, f), os.path.join(dest_dir_style, f))

print(f"✅ Copié {len(sample_files)} fichiers dans {dest_dir}")


✅ Copié 20 fichiers dans /home/phusson/Bureau/Erriep56.github.io/images


In [2]:
import os
dest_dir = f'{os.getcwd()}/images'
print(dest_dir)

/home/phusson/Bureau/Erriep56.github.io/images
